### Buy and Hold
1. 구매 후 일정 기간이 지나면 판매

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../../csv/AAPL.csv', index_col='Date')
df.head(1)

In [ ]:
# 결측치와 무한대 값들이 존재하는가?
df.isin([np.nan, np.inf, -np.inf]).any(axis=0)

In [ ]:
df.loc[:, (df.isin([np.nan, np.inf, -np.inf]).any(axis=0))]

In [ ]:
# 결측치와 무한대의 값을 제외한 데이터
df = df.loc[~(df.isin([np.nan, np.inf, -np.inf]).any(axis=1))]

In [ ]:
# 수정종가를 기준으로 구매와 판매를 하여 수익율 계산
price_df = df[['Adj Close']]

In [ ]:
price_df.plot()

In [ ]:
# 일별 수익율
# (오늘의 종가 - 전날의 종가) / 전날의 종가
# case1
(price_df['Adj Close'] - price_df['Adj Close'].shift(1)) \
    / price_df['Adj Close'].shift(1)
# case2
price_df['Adj Close'].diff() / price_df['Adj Close'].shift()
# case3 
price_df['daily_rtn'] = price_df['Adj Close'].pct_change()

In [ ]:
price_df.head(4)

In [ ]:
test_df = price_df.iloc[6000:8000]

In [ ]:
# 수익율 계산
test_df.iloc[-1, 0] / test_df.iloc[0, 0]

In [ ]:
# 일별 수익율 가지고 수익율 계산

# 초기 수익율 1로 지정
rtn = 1

# (일별 수익율 + 1) * rtn -> 수익율
for i in range(len(test_df)):
    # i는 인덱스의 위치값
    # rtn = (test_df.iloc[i, 1] + 1) * rtn
    rtn *= (test_df.iloc[i, 1] + 1)
print(rtn)

In [ ]:
# 누적곱을 구하는 함수
(1 + test_df['daily_rtn']).cumprod()